In [172]:
import numpy as np
import pandas as pd

In [173]:
!pip3 install pyvi

In [174]:
from pyvi import ViPosTagger, ViTokenizer

In [175]:
data_train = pd.read_csv('./data_train.csv')

In [176]:
X = data_train[['Review']].sort_index()
y = data_train['Label'].fillna(0.)
print(X.shape[0])

4140


In [177]:
englishwords = pd.read_json("./englishwords.json")
englishwords.set_index("english", inplace = True)
shortform = pd.read_json("./shortform.json")
shortform.set_index("short", inplace = True)

In [178]:
# Loại bỏ kí tự đặc biệt và chuyển thành chữ cái viết thường
import re 

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ' '))
    
    #Loại bỏ các chữ cái lặp nhau
    length = len(text)
    char = 0
    while char <length-1:
        if text[char] == text[char+1]:
            text = text[:char]+text[char+1:]
            #print(text)
            length-=1
            continue
        char+=1
    #loại bỏ số  và các chữ cái lạ:
    digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ(?::|;|=)(?:-)?(?:\)|\(|D|P)_ '
    text = ''.join([i for i in text if i in digits_and_characters])

    text_split = text.split()
    #print(text_split)
    for i in range(len(text_split)):
      t = text_split[i]
      if t in englishwords.index:
        text_split[i] = str(englishwords.loc[t, "vietnamese"])
      elif t in shortform.index:
        text_split[i] = str(shortform.loc[t,"long"]) 
    #print(text_split)
    text = ' '.join(text_split)
    return text


In [179]:
def clean(data):
  for i in data.index:
    data.loc[i, "Review"] = ViTokenizer.tokenize(preprocessor(data.loc[i, "Review"]))

In [180]:
def splitSentence(text):
  list_of_sentences = text.split('.')
  return list_of_sentences
print(splitSentence("dùng khá ổn. Cực kì ủng hộ.Ilove you"))
a,b, c = splitSentence("dùng khá ổn. Cực kì ủng hộ.Ilove you")
print(a)
print(b)
print(c)

['dùng khá ổn', ' Cực kì ủng hộ', 'Ilove you']
dùng khá ổn
 Cực kì ủng hộ
Ilove you


In [181]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [182]:
with open('./stopwords_vn_dash.txt') as f:
    stopwords_vn = f.read().splitlines()

In [183]:
clean(X_train)
print(X_train.loc[3444])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Review    def
Name: 3444, dtype: object


In [184]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(stop_words = stopwords_vn, min_df = 2, ngram_range= (1,2))
X_train_bow = bow.fit_transform(X_train["Review"])
print(X_train_bow.shape)
print(bow.get_feature_names())

(2898, 13909)
['_mình', 'about', 'about the', 'ac', 'ace', 'ad', 'adayroi', 'ae', 'aen', 'aeon', 'aeon không', 'aeon mal', 'after', 'ahihi', 'ai_ngờ', 'al', 'al fresco', 'al italian', 'al không', 'al the', 'almaz', 'alo', 'alo bàn', 'also', 'always', 'amount', 'amount of', 'an', 'an hành_tây', 'an_toàn', 'an_toàn thực_phẩm', 'an_tâm', 'and', 'and tasty', 'and the', 'and there', 'and they', 'and this', 'anh_chị', 'anh_chị không', 'anh_chị nhân_viên', 'anh_chị nv', 'anh_chị phục_vụ', 'anh_chị đi', 'anh_chị_em', 'anh_chị_em công_sở', 'anh_em', 'are', 'are realy', 'aroi', 'around', 'as', 'at', 'at al', 'authentic', 'avalon', 'away', 'ba_chỉ', 'ba_chỉ bò', 'ba_chỉ heo', 'ba_chỉ ngon', 'ba_chỉ nướng', 'ba_chỉ nấm', 'back', 'bacon', 'bad', 'baguete', 'bal', 'bami', 'ban', 'ban_công', 'ban_ngày', 'ban_ngày không', 'ban_đầu', 'ban_đầu không', 'ban_đầu quán', 'banh', 'bao', 'bao gìơ', 'bao gọi', 'bao không', 'bao kim_sa', 'bao quán', 'bao đi', 'bao_bì', 'bao_cấp', 'bao_gồm', 'bao_quát', 'baoh', 

In [185]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = stopwords_vn, min_df = 2, ngram_range= (1,2))
# Feed the tf-idf Vectorizer with twits using fit_transform()
X_train_tfidf = tfidf.fit_transform(X_train["Review"])

# Formatting the number to 2 digits after the decimal point by showing on this notebook
np.set_printoptions(precision=2)
# To print array in one line
np.set_printoptions(linewidth=np.inf)
print(tfidf.get_feature_names())
print(X_train_tfidf.shape)

['_mình', 'about', 'about the', 'ac', 'ace', 'ad', 'adayroi', 'ae', 'aen', 'aeon', 'aeon không', 'aeon mal', 'after', 'ahihi', 'ai_ngờ', 'al', 'al fresco', 'al italian', 'al không', 'al the', 'almaz', 'alo', 'alo bàn', 'also', 'always', 'amount', 'amount of', 'an', 'an hành_tây', 'an_toàn', 'an_toàn thực_phẩm', 'an_tâm', 'and', 'and tasty', 'and the', 'and there', 'and they', 'and this', 'anh_chị', 'anh_chị không', 'anh_chị nhân_viên', 'anh_chị nv', 'anh_chị phục_vụ', 'anh_chị đi', 'anh_chị_em', 'anh_chị_em công_sở', 'anh_em', 'are', 'are realy', 'aroi', 'around', 'as', 'at', 'at al', 'authentic', 'avalon', 'away', 'ba_chỉ', 'ba_chỉ bò', 'ba_chỉ heo', 'ba_chỉ ngon', 'ba_chỉ nướng', 'ba_chỉ nấm', 'back', 'bacon', 'bad', 'baguete', 'bal', 'bami', 'ban', 'ban_công', 'ban_ngày', 'ban_ngày không', 'ban_đầu', 'ban_đầu không', 'ban_đầu quán', 'banh', 'bao', 'bao gìơ', 'bao gọi', 'bao không', 'bao kim_sa', 'bao quán', 'bao đi', 'bao_bì', 'bao_cấp', 'bao_gồm', 'bao_quát', 'baoh', 'bar', 'baskin

In [186]:
clean(X_test)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [187]:
X_test_split = pd.DataFrame(columns = ["Sentence","Origin"])
print(X_test_split)
for i in X_test.index:
  los = splitSentence(X_test.loc[i,"Review"])
  for j in range (len(los)):
    new_row = {"Sentence": los[j], "Origin": i}
    X_test_split = X_test_split.append(new_row, ignore_index= True)
print(X_test_split.head())

Empty DataFrame
Columns: [Sentence, Origin]
Index: []
                                            Sentence Origin
0  mình mới ăn_ở đây lần đầu từ lúc quán mới mở đ...   3487
1  vị_trí thuận_tiện ngay trung_tâm thương bài_tr...   1964
2                     chả cốm không nơi nào ngon_vậy   1582
3  không_gian tương đoi viên phục_vụ cũng khá luc...    296
4  trời mình đặt bàn trước là phòng riêng ở trên ...    149


In [188]:
X_test_bow = bow.transform(X_test_split["Sentence"])
X_test_tfidf = tfidf.transform(X_test_split["Sentence"])

In [189]:
from sklearn.naive_bayes import MultinomialNB
nbc = MultinomialNB()

In [190]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [191]:
nbc.fit(X_train_bow.toarray(), y_train)
y_pred = nbc.predict(X_test_bow.toarray())
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()

In [192]:
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7705314009661836
0.5677711233090234


In [193]:
nbc.fit(X_train_tfidf.toarray(), y_train)
y_pred = nbc.predict(X_test_tfidf.toarray())
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()

In [194]:
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7600644122383253
0.5401990634243752


In [195]:
from sklearn.linear_model import LogisticRegression


In [196]:
lor = LogisticRegression()
lor.fit(X_train_bow.toarray(), y_train)
y_pred = lor.predict(X_test_bow.toarray())
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7383252818035426
0.5799312968430616


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [197]:
lor.fit(X_train_tfidf.toarray(), y_train)
y_pred = lor.predict(X_test_tfidf.toarray())
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))
print(classification_report(y_test, y_pred))

0.7665056360708534
0.5525800311863515
              precision    recall  f1-score   support

        -1.0       0.78      0.82      0.80       520
         0.0       1.00      0.01      0.02       159
         1.0       0.76      0.93      0.83       563

    accuracy                           0.77      1242
   macro avg       0.84      0.59      0.55      1242
weighted avg       0.80      0.77      0.72      1242



In [198]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth = None, random_state = 0)

In [199]:
rfc.fit(X_train_bow.toarray(), y_train)
y_pred = rfc.predict(X_test_bow.toarray())
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7479871175523349
0.5451650822964632


In [200]:
rfc.fit(X_train_tfidf.toarray(), y_train)
y_pred = rfc.predict(X_test_tfidf.toarray())
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7528180354267311
0.5472457719359327


In [201]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()

In [202]:
mlp.fit(X_train_bow, y_train)
y_pred = mlp.predict(X_test_bow)
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.76      0.77      0.77       520
         0.0       0.24      0.16      0.19       159
         1.0       0.79      0.86      0.83       563

    accuracy                           0.73      1242
   macro avg       0.60      0.60      0.60      1242
weighted avg       0.71      0.73      0.72      1242



In [203]:
mlp.fit(X_train_tfidf, y_train)
y_pred = mlp.predict(X_test_tfidf)
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.77      0.79      0.78       520
         0.0       0.29      0.14      0.19       159
         1.0       0.80      0.90      0.85       563

    accuracy                           0.76      1242
   macro avg       0.62      0.61      0.60      1242
weighted avg       0.72      0.76      0.73      1242



In [204]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler(with_mean= False)
X_train_bow = ss.fit_transform(X_train_bow)
X_test_bow = ss.transform(X_test_bow)
X_train_tfidf = ss.fit_transform(X_train_tfidf)
X_test_tfidf  = ss.transform(X_test_tfidf)

In [205]:
from sklearn.svm import SVC
clf_svc = SVC(C = 1.)



In [206]:
clf_svc.fit(X_train_bow, y_train)
y_pred = clf_svc.predict(X_test_bow)
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.77      0.45      0.56       520
         0.0       0.00      0.00      0.00       159
         1.0       0.56      0.94      0.71       563

    accuracy                           0.61      1242
   macro avg       0.44      0.46      0.42      1242
weighted avg       0.58      0.61      0.56      1242



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [207]:
clf_svc.fit(X_train_tfidf, y_train)
y_pred = clf_svc.predict(X_test_tfidf)
X_test_split["Predict"] = y_pred
X_test_split
X_test_split.groupby("Origin").sum()
y_pred = X_test_split["Predict"].tolist()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.71      0.74      0.73       520
         0.0       0.00      0.00      0.00       159
         1.0       0.68      0.85      0.75       563

    accuracy                           0.69      1242
   macro avg       0.46      0.53      0.49      1242
weighted avg       0.61      0.69      0.65      1242



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [208]:
import pickle

In [210]:
file_name = "nbc_model.sav"
pickle.dump(nbc, open(file_name, 'wb'))

In [214]:
loaded_model = pickle.load(open(file_name, 'rb'))
print(loaded_model)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
